In [2]:
import mdptoolbox.mdp as mdp
import numpy as np
import matplotlib.pyplot as plt

In [3]:
# El siguiente código lee el goal y el mapa del fichero

def lee_mapa(fichero):
    with open(fichero,'r') as archivo:
        lineas = archivo.readlines()
    numeros = [float(numero) for numero in lineas[0].split()]
    lineas.pop(0)
    lineas.reverse()
    matriz = []
    for linea in lineas:
        fila = [int(caracter) for caracter in linea.strip()]
        matriz.append(fila)
    return np.array(matriz),(numeros[0],numeros[1])

FileNotFoundError: [Errno 2] No such file or directory: 'data/map1.txt'